## Obama-Clinton 2008 Democratic Primaries

Import everything

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import train_test_split, GridSearchCV
import statsmodels.formula.api as smf

from patsy import dmatrices, dmatrix, build_design_matrices

import math

In [2]:
# This makes it so that the pandas dataframes don't get truncated horizontally.
pd.options.display.max_columns = 200

Bring in the data

In [3]:
elect_df = pd.read_csv("Obama.csv")

Split training & test

In [4]:
elect_df["ElectionDate"] = pd.to_datetime(elect_df["ElectionDate"])

All of the data before February 19, 2008 is our training set and all of the data after that point is our testing set.

In [5]:
elect_df_train = elect_df[elect_df["ElectionDate"] < pd.to_datetime("2/19/2008")].copy()

In [6]:
elect_df_test = elect_df[elect_df["ElectionDate"] >= pd.to_datetime("2/19/2008")].copy()

Create more variables

In [15]:
elect_df_train["Obama_margin"] = elect_df_train["Obama"] - elect_df_train["Clinton"]
elect_df_train["Obama_margin_percent"] = elect_df_train["Obama_margin"] / elect_df_train["TotalVote"]
elect_df_train["Obama_wins"] = elect_df_train["Obama_margin"] > 0

In [16]:
def summarize_dataframe(df):
    """Summarize a dataframe, and report missing values."""
    missing_values = pd.DataFrame({'Variable Name': df.columns,
                                   'Data Type': df.dtypes,
                                   'Missing Values': df.isnull().sum(),
                                   'Unique Values': [df[name].nunique() for name in df.columns]}
                                 ).set_index('Variable Name')
    with pd.option_context("display.max_rows", 1000):
        display(pd.concat([missing_values, df.describe(include='all').transpose()], axis=1).fillna(""))

In [17]:
summarize_dataframe(elect_df)

/var/folders/x9/19r3j4q963qdgn68y0lk16d00000gn/T/ipykernel_32037/2697573813.py:9: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  display(pd.concat([missing_values, df.describe(include='all').transpose()], axis=1).fillna(""))


,Data Type,Missing Values,Unique Values,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
County,object,0,1691,2868.0,1691,Washington,30,NaT,NaT,,,,,,,
State,object,0,46,2868.0,46,TX,251,NaT,NaT,,,,,,,
Region,object,0,4,2868.0,4,South,1419,NaT,NaT,,,,,,,
FIPS,int64,0,2868,2868.0,,,,NaT,NaT,31029.039052,15411.398662,1001.0,18102.5,30110.0,46123.5,56045.0
ElectionDate,datetime64[ns],0,18,2868.0,18,2008-02-05 00:00:00,1128,2008-01-03,2008-06-03,,,,,,,
ElectionType,object,0,2,2868.0,2,Primary,2558,NaT,NaT,,,,,,,
TotalVote,float64,1131,1490,1737.0,,,,NaT,NaT,12863.7962,53071.884245,13.0,732.0,2330.0,6418.0,1413869.0
Clinton,float64,1131,1338,1737.0,,,,NaT,NaT,5973.738054,25402.029497,4.0,329.0,1106.0,3200.0,771700.0
Obama,float64,1131,1314,1737.0,,,,NaT,NaT,6178.122625,27477.231157,4.0,254.0,878.0,2749.0,743686.0
MalesPer100Females,float64,0,391,2868.0,,,,NaT,NaT,99.081729,9.276792,76.2,94.9,97.6,100.5,200.9


Split into smaller sets & validation

In [18]:
elect_df_smaller_train, elect_df_validation = train_test_split(elect_df_train, test_size=.25, random_state=201)

In [19]:
elect_df_smaller_train

,County,State,Region,FIPS,ElectionDate,ElectionType,TotalVote,Clinton,Obama,MalesPer100Females,AgeBelow35,Age35to65,Age65andAbove,White,Black,Asian,AmericanIndian,Hawaiian,Hispanic,HighSchool,Bachelors,Poverty,IncomeAbove75K,MedianIncome,AverageIncome,UnemployRate,ManfEmploy,SpeakingNonEnglish,Medicare,MedicareRate,SocialSecurity,SocialSecurityRate,RetiredWorkers,Disabilities,DisabilitiesRate,Homeowner,SameHouse1995and2000,Pop,PopDensity,LandArea,FarmArea,Obama_margin,Obama_margin_percent,Obama_wins
1677,Rappahannock,VA,South,51157,2008-02-12,Primary,1300.0,515.0,774.0,97.7,38.4,46.3,15.1,93.4,5.4,0.2,0.2,0.0,1.9,76.0,22.9,6.6,25.6,52195.0,33902.0,2.3,3.252033,3.0,1552.0,21345.0,1485.0,20424.0,1035.0,95.0,1307.0,75.2,66.1,7203,27.0,267.0,78.0,259.0,0.199231,True
594,Greene,GA,South,13133,2008-02-05,Primary,1822.0,518.0,1245.0,94.7,46.4,37.5,16.1,58.8,40.3,0.4,0.3,0.1,3.5,70.1,17.6,18.2,18.8,36104.0,27818.0,5.9,6.946108,3.6,2938.0,18722.0,3655.0,23291.0,2335.0,535.0,3409.0,76.4,56.4,15534,40.0,406.0,52.0,727.0,0.399012,True
642,Pike,GA,South,13231,2008-02-05,Primary,1322.0,639.0,612.0,102.1,50.0,39.3,10.8,86.5,12.5,0.5,0.2,0.0,1.4,75.3,14.0,10.5,18.5,47796.0,25669.0,4.6,10.652572,2.9,2206.0,13678.0,2680.0,16617.0,1715.0,279.0,1730.0,81.6,57.4,16801,76.9,219.0,44.0,-27.0,-0.020424,False
1200,Johnston,OK,South,40069,2008-02-05,Primary,2014.0,1181.0,403.0,96.3,45.0,39.0,15.9,77.7,2.0,0.3,15.5,0.0,3.1,69.1,13.3,16.1,7.5,28405.0,21715.0,4.3,14.529221,3.4,2029.0,19778.0,2505.0,24418.0,1380.0,439.0,4279.0,73.9,56.5,10436,16.2,658.0,326.0,-778.0,-0.386296,False
81,Scott,IA,Midwest,19163,2008-01-03,Caucuses,350.0,103.0,169.0,96.6,46.5,41.4,12.1,89.9,6.2,1.8,0.3,0.0,4.5,86.3,24.9,12.3,20.5,48884.0,35018.0,3.8,11.743811,5.6,22900.0,14224.0,26525.0,16475.0,17260.0,3083.0,1915.0,70.6,54.0,162621,355.1,468.0,229.0,66.0,0.188571,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,Douglas,MN,Midwest,27041,2008-02-05,Caucuses,771.0,362.0,399.0,99.3,44.0,37.8,18.3,98.5,0.4,0.4,0.3,0.0,0.9,85.6,17.3,7.5,13.6,42640.0,30121.0,3.8,11.195793,3.2,7237.0,20596.0,8265.0,23522.0,5690.0,469.0,1335.0,77.2,60.7,35467,55.9,720.0,273.0,37.0,0.047990,True
1377,Caldwell,LA,South,22021,2008-02-09,Primary,897.0,441.0,313.0,103.8,47.5,38.3,14.2,81.6,17.7,0.2,0.4,0.0,1.6,65.4,8.8,20.6,8.1,31957.0,21050.0,3.8,2.721088,3.3,1864.0,17647.0,2040.0,19313.0,1025.0,470.0,4449.0,79.2,60.5,10615,20.1,541.0,61.0,-128.0,-0.142698,False
1536,Grant,WA,West,53025,2008-02-09,Primary,5003.0,2615.0,2186.0,105.2,54.2,34.4,11.5,95.1,1.1,1.1,1.4,0.1,33.8,72.2,13.7,16.2,13.1,38627.0,22538.0,6.6,9.194838,28.3,11063.0,13620.0,12605.0,15518.0,8115.0,1735.0,2136.0,66.7,50.0,82612,30.8,2791.0,1074.0,-429.0,-0.085749,False
1577,Washington,ME,Northeast,23029,2008-02-10,Caucuses,55.0,20.0,35.0,95.6,39.6,42.7,17.8,93.8,0.4,0.4,4.6,0.0,1.1,79.9,14.7,17.4,7.8,29721.0,25094.0,7.4,8.664908,5.4,7501.0,22426.0,8570.0,25622.0,5270.0,1270.0,3797.0,77.7,66.7,33288,13.0,3255.0,152.0,15.0,0.272727,True


Impute missing data

In [20]:
imputer_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer_zero = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)

In [21]:
continuous_mean = ['MalesPer100Females',
                   'AgeBelow35', 'Age35to65','Age65andAbove']

continuous_zero = ['White', 
                   'Black',
                   'Asian',
                   'AmericanIndian',
                   'Hawaiian',
                  'Hispanic']

In order to impute anything, we need to `fit` the imputer. The imputer needs to know what the mean of all of the existing data is, so we give it the data set, and we let it learn the mean. After it has been fitted, we can use it to actually transform the data.

In [25]:
imputer_mean.fit(elect_df_train[continuous_mean])
imputer_mean.fit(elect_df_smaller_train[continuous_mean])
elect_df_train[continuous_mean] = imputer_mean.transform(elect_df_train[continuous_mean])
elect_df_smaller_train[continuous_mean] = imputer_mean.transform(elect_df_smaller_train[continuous_mean])
elect_df_validation[continuous_mean] = imputer_mean.transform(elect_df_validation[continuous_mean])

/Users/locolaurel/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/Users/locolaurel/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [26]:
imputer_zero.fit(elect_df_train[continuous_zero])
imputer_zero.fit(elect_df_smaller_train[continuous_zero])
elect_df_train[continuous_zero] = imputer_zero.transform(elect_df_train[continuous_zero])
elect_df_smaller_train[continuous_zero] = imputer_zero.transform(elect_df_smaller_train[continuous_zero])
elect_df_validation[continuous_zero] = imputer_zero.transform(elect_df_validation[continuous_zero])

/Users/locolaurel/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
/Users/locolaurel/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [27]:
imputer_mean.fit(elect_df_train.loc[:, "HighSchool":"FarmArea"])
imputer_mean.fit(elect_df_smaller_train.loc[:, "HighSchool":"FarmArea"])
elect_df_train.loc[:, "HighSchool":"FarmArea"] = imputer_mean.transform(elect_df_train.loc[:, "HighSchool":"FarmArea"])
elect_df_smaller_train.loc[:, "HighSchool":"FarmArea"] = imputer_mean.transform(elect_df_smaller_train.loc[:, "HighSchool":"FarmArea"])
elect_df_validation.loc[:, "HighSchool":"FarmArea"] = imputer_mean.transform(elect_df_validation.loc[:, "HighSchool":"FarmArea"])

/Users/locolaurel/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/Users/locolaurel/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1835: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)


Accuracy function

In [28]:
def accuracy(y_true, y_pred):
    """Function that returns a table showing RMSE and MAE."""
    acc_df = pd.DataFrame(data = {"RMSE": [mean_squared_error(y_true, y_pred)**(1/2)],
                                  "MAE": [mean_absolute_error(y_true, y_pred)],
                                  "R^2": [r2_score(y_true, y_pred)]})
    display(acc_df.style.hide_index())

Training model:

In [29]:
variables2 = [variable for variable in elect_df_smaller_train.columns if variable not in ['FIPS', 'County', 'State', 'ElectionDate', 'TotalVote', 'Obama_margin', 'Obama_margin_percent', 'Obama_wins', 'Clinton', 'Obama','Region','ElectionType']]



In [30]:
formula2 = 'Obama_margin_percent ~ ElectionType' +  ' + Region*' + "(standardize(" + ") + standardize(".join(variables2[1:]) + "))"
                                                                      

In [31]:
formula2

'Obama_margin_percent ~ ElectionType + Region*(standardize(AgeBelow35) + standardize(Age35to65) + standardize(Age65andAbove) + standardize(White) + standardize(Black) + standardize(Asian) + standardize(AmericanIndian) + standardize(Hawaiian) + standardize(Hispanic) + standardize(HighSchool) + standardize(Bachelors) + standardize(Poverty) + standardize(IncomeAbove75K) + standardize(MedianIncome) + standardize(AverageIncome) + standardize(UnemployRate) + standardize(ManfEmploy) + standardize(SpeakingNonEnglish) + standardize(Medicare) + standardize(MedicareRate) + standardize(SocialSecurity) + standardize(SocialSecurityRate) + standardize(RetiredWorkers) + standardize(Disabilities) + standardize(DisabilitiesRate) + standardize(Homeowner) + standardize(SameHouse1995and2000) + standardize(Pop) + standardize(PopDensity) + standardize(LandArea) + standardize(FarmArea))'

In [32]:
y_linear2, X_linear2 = dmatrices(formula2, elect_df_smaller_train, return_type="dataframe")

In [33]:
X_validation_linear2 = build_design_matrices([X_linear2.design_info], elect_df_validation, return_type="dataframe")[0]

In [34]:
alphas = np.logspace(-5,5,30)

You might get a few warnings below. Ignore them.

In [35]:
%%time
coefs = []
for a in alphas:
    lm_ridge = Ridge(alpha=a, fit_intercept=False, max_iter=1000000)
    lm_ridge.fit(X_linear2, y_linear2)
    coefs.append(lm_ridge.coef_[0])

CPU times: user 343 ms, sys: 11.5 ms, total: 355 ms
Wall time: 229 ms


In [36]:
gs_ridge2 = GridSearchCV(Ridge(fit_intercept=False, max_iter=1000000), {"alpha": alphas}, scoring='neg_mean_squared_error', cv=5)

In [37]:
%%time
gs_ridge2.fit(X_linear2, y_linear2)

CPU times: user 2.55 s, sys: 25.9 ms, total: 2.57 s
Wall time: 1.37 s


GridSearchCV(cv=5, estimator=Ridge(fit_intercept=False, max_iter=1000000),
             param_grid={'alpha': array([1.00000000e-05, 2.21221629e-05, 4.89390092e-05, 1.08263673e-04,
       2.39502662e-04, 5.29831691e-04, 1.17210230e-03, 2.59294380e-03,
       5.73615251e-03, 1.26896100e-02, 2.80721620e-02, 6.21016942e-02,
       1.37382380e-01, 3.03919538e-01, 6.72335754e-01, 1.48735211e+00,
       3.29034456e+00, 7.27895384e+00, 1.61026203e+01, 3.56224789e+01,
       7.88046282e+01, 1.74332882e+02, 3.85662042e+02, 8.53167852e+02,
       1.88739182e+03, 4.17531894e+03, 9.23670857e+03, 2.04335972e+04,
       4.52035366e+04, 1.00000000e+05])},
             scoring='neg_mean_squared_error')

In [38]:
gs_ridge_pred2 = gs_ridge2.predict(X_validation_linear2)

In [39]:
accuracy(gs_ridge_pred2, elect_df_validation["Obama_margin_percent"])

RMSE,MAE,R^2
0.158079,0.115398,0.650562


In [41]:
def get_linear_importances(model, data):
    # This section is to handle both LASSO and ridge.
    if len(model.coef_) == 1:
        coefficients = np.concatenate((model.coef_[0], [model.intercept_]))
    else:
        coefficients = np.concatenate((model.coef_, model.intercept_))
    coef_df = pd.DataFrame({'Importance': coefficients}, index=data.columns.append(pd.Index(["Model Intercept"])))
    return coef_df.reindex(coef_df.Importance.abs().sort_values(ascending=False).index)

In [42]:
get_linear_importances(gs_ridge2.best_estimator_, X_linear2)

,Importance
Region[T.South],-0.185802
Region[T.Northeast]:standardize(SameHouse1995and2000),-0.139900
Region[T.Northeast],-0.108427
standardize(Black),0.107146
Region[T.Northeast]:standardize(Age35to65),0.105192
...,...
standardize(PopDensity),0.001800
Region[T.Northeast]:standardize(Bachelors),-0.001563
standardize(ManfEmploy),0.001040
Region[T.South]:standardize(RetiredWorkers),0.000269


Model to submit:

In [43]:
variables = [variable for variable in elect_df_train.columns if variable not in ['FIPS', 'County', 'State', 'ElectionDate', 'TotalVote', 'Obama_margin', 'Obama_margin_percent', 'Obama_wins', 'Clinton', 'Obama','MalesPer100Females','Hawaiian','Region','ElectionType']]

In [44]:
formula = 'Obama_margin_percent ~ (Region*ElectionType)' +  ' + Region*' + "(standardize(" + ") + standardize(".join(variables[1:]) + "))"

In [45]:
formula

'Obama_margin_percent ~ (Region*ElectionType) + Region*(standardize(Age35to65) + standardize(Age65andAbove) + standardize(White) + standardize(Black) + standardize(Asian) + standardize(AmericanIndian) + standardize(Hispanic) + standardize(HighSchool) + standardize(Bachelors) + standardize(Poverty) + standardize(IncomeAbove75K) + standardize(MedianIncome) + standardize(AverageIncome) + standardize(UnemployRate) + standardize(ManfEmploy) + standardize(SpeakingNonEnglish) + standardize(Medicare) + standardize(MedicareRate) + standardize(SocialSecurity) + standardize(SocialSecurityRate) + standardize(RetiredWorkers) + standardize(Disabilities) + standardize(DisabilitiesRate) + standardize(Homeowner) + standardize(SameHouse1995and2000) + standardize(Pop) + standardize(PopDensity) + standardize(LandArea) + standardize(FarmArea))'

In [46]:
y_linear, X_linear = dmatrices(formula, elect_df_train, return_type="dataframe")

In [47]:
X_validation_linear = build_design_matrices([X_linear.design_info], elect_df_test, return_type="dataframe")[0]

In [48]:
alphas = np.logspace(-5,10, 31)

You might get a few warnings below. Ignore them.

In [49]:
%%time
coefs = []
for a in alphas:
    lm_ridge = Ridge(alpha=a, fit_intercept=False, max_iter=1000000)
    lm_ridge.fit(X_linear, y_linear)
    coefs.append(lm_ridge.coef_[0])

CPU times: user 315 ms, sys: 6.82 ms, total: 322 ms
Wall time: 226 ms


In [50]:
gs_ridge = GridSearchCV(Ridge(fit_intercept=False, max_iter=1000000), {"alpha": alphas}, scoring='neg_mean_squared_error', cv=5)

In [51]:
%%time
gs_ridge.fit(X_linear, y_linear)

CPU times: user 2.62 s, sys: 46.1 ms, total: 2.67 s
Wall time: 1.44 s


GridSearchCV(cv=5, estimator=Ridge(fit_intercept=False, max_iter=1000000),
             param_grid={'alpha': array([1.00000000e-05, 3.16227766e-05, 1.00000000e-04, 3.16227766e-04,
       1.00000000e-03, 3.16227766e-03, 1.00000000e-02, 3.16227766e-02,
       1.00000000e-01, 3.16227766e-01, 1.00000000e+00, 3.16227766e+00,
       1.00000000e+01, 3.16227766e+01, 1.00000000e+02, 3.16227766e+02,
       1.00000000e+03, 3.16227766e+03, 1.00000000e+04, 3.16227766e+04,
       1.00000000e+05, 3.16227766e+05, 1.00000000e+06, 3.16227766e+06,
       1.00000000e+07, 3.16227766e+07, 1.00000000e+08, 3.16227766e+08,
       1.00000000e+09, 3.16227766e+09, 1.00000000e+10])},
             scoring='neg_mean_squared_error')

In [52]:
gs_ridge_pred = gs_ridge.predict(X_validation_linear)

In [53]:
def get_linear_importances(model, data):
    # This section is to handle both LASSO and ridge.
    if len(model.coef_) == 1:
        coefficients = np.concatenate((model.coef_[0], [model.intercept_]))
    else:
        coefficients = np.concatenate((model.coef_, model.intercept_))
    coef_df = pd.DataFrame({'Importance': coefficients}, index=data.columns.append(pd.Index(["Model Intercept"])))
    return coef_df.reindex(coef_df.Importance.abs().sort_values(ascending=False).index)

In [54]:
get_linear_importances(gs_ridge.best_estimator_, X_linear)

,Importance
Region[T.Northeast]:ElectionType[T.Primary],-0.109740
Region[T.South]:standardize(Black),0.096173
standardize(Black),0.086440
Region[T.South],-0.082756
Region[T.South]:ElectionType[T.Primary],-0.082756
...,...
standardize(PopDensity),0.000671
Region[T.Northeast]:standardize(Disabilities),-0.000478
Region[T.West]:standardize(Medicare),0.000228
Region[T.Northeast]:standardize(SocialSecurityRate),0.000088


In [55]:
bm_all_fit_pred = gs_ridge_pred

In [ ]:
submission_df = pd.DataFrame(bm_all_fit_pred, index=elect_df_test.index, columns=['Obama_margin_percent'])

In [ ]:
submission_df

In [ ]:
submission_df.to_csv("ObamaSubmission.csv", index_label='Row')